In [1]:
%pip install selenium webdriver-manager -qqq

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
import os

# Set up Chrome options
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Launch browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [3]:
NUM_ITEMS_TO_DOWNLOAD = 5
BASE_URL = "https://www.courtlistener.com/?type=oa&q=&type=oa&order_by=score%20desc"
OUTPUT_DIR = "courtlistener_mp3s"
os.makedirs(OUTPUT_DIR, exist_ok=True)

driver.get(BASE_URL)
time.sleep(3)

In [4]:
case_links = []
case_elements = driver.find_elements(By.CSS_SELECTOR, "h3.bottom.serif a")

for case in case_elements:
    href = case.get_attribute("href")
    if href and "/audio/" in href:
        case_links.append(href)

# Trim to how many we want
case_links = case_links[:NUM_ITEMS_TO_DOWNLOAD]

print(f"Found {len(case_links)} case pages.")
case_links


Found 5 case pages.


['https://www.courtlistener.com/audio/98513/united-states-v-gregory/?type=oa&type=oa&q=&order_by=score+desc',
 'https://www.courtlistener.com/audio/98515/united-states-v-arturs-spila/?type=oa&type=oa&q=&order_by=score+desc',
 'https://www.courtlistener.com/audio/98521/united-states-v-roger-snake/?type=oa&type=oa&q=&order_by=score+desc',
 'https://www.courtlistener.com/audio/98523/united-states-v-evelio-santana/?type=oa&type=oa&q=&order_by=score+desc',
 'https://www.courtlistener.com/audio/98522/united-states-v-paul-williams-anti/?type=oa&type=oa&q=&order_by=score+desc']

In [ ]:
from urllib.parse import urlparse
import re
import time

for i, case_url in enumerate(case_links):
    driver.get(case_url)
    time.sleep(2)  # Wait for the case page to load

    try:
        # Extract case name from the URL
        path_parts = urlparse(case_url).path.strip('/').split('/')
        case_slug = path_parts[-1]  # e.g., "united-states-v-gregory"
        filename_base = re.sub(r'[^\w\-]', '', case_slug)
        filename = f"{filename_base}.mp3"
        filepath = os.path.join(OUTPUT_DIR, filename)

        # Find and download the MP3
        mp3_button = driver.find_element(By.LINK_TEXT, "Download MP3")
        mp3_url = mp3_button.get_attribute("href")
        mp3_data = requests.get(mp3_url).content

        with open(filepath, "wb") as f:
            f.write(mp3_data)

        print(f"✅ Downloaded {filename}")
    except Exception as e:
        print(f"❌ Failed to download from {case_url}: {e}")


❌ Failed to download from https://www.courtlistener.com/audio/98513/united-states-v-gregory/?type=oa&type=oa&q=&order_by=score+desc: Could not find case title on page
❌ Failed to download from https://www.courtlistener.com/audio/98515/united-states-v-arturs-spila/?type=oa&type=oa&q=&order_by=score+desc: Could not find case title on page
❌ Failed to download from https://www.courtlistener.com/audio/98521/united-states-v-roger-snake/?type=oa&type=oa&q=&order_by=score+desc: Could not find case title on page
❌ Failed to download from https://www.courtlistener.com/audio/98523/united-states-v-evelio-santana/?type=oa&type=oa&q=&order_by=score+desc: Could not find case title on page
❌ Failed to download from https://www.courtlistener.com/audio/98522/united-states-v-paul-williams-anti/?type=oa&type=oa&q=&order_by=score+desc: Could not find case title on page


In [6]:
driver.quit()